# Natural Vibration of a Simply Supported Beam

In [1]:
import texttable
import numpy as np


def print_modal_table(m_eff, freq, tol=10):
    tableObj = texttable.Texttable()
    rows = [["", "mx", "my", "mz", "freq"]]
    for i in range(freq.shape[0]):
        if np.max(m_eff[i]) > tol:
            rows.append([i + 1, m_eff[i, 0], m_eff[i, 1], m_eff[i, 2], freq[i]])
    tableObj.add_rows(rows)
    print(tableObj.draw())

In [2]:
# all units in kN and cm
L = 150.0  # length of the console [cm]
F = 1.0  # value of the vertical load at the free end [kN]
E = 21000.0  # Young's modulus [kN/cm3]
nu = 0.3  # Poisson's ratio [-]
w, h = 5.0, 15.0  # width and height of the rectangular cross section
g = 9.81  # gravitational acceleration [m/s2]
density = 7750.0 * 1e-6  # mass density [g/cm3]
nElem = 20  # number of subdivisons to use

In [3]:
A = w * h  # area
Iy = w * h**3 / 12  # second moment of inertia around the y axis
Iz = h * w**3 / 12  # second moment of inertia around the z axis
Ix = Iy + Iz  # torsional inertia
weight = density * 1e-3 * g  # [kN/cm3]
dpa = density * A  # density per area [g/cm]
wpa = weight * A  # weight per area [kN/cm]
mass = L * dpa  # total mass

## Vinbration Analysis

In [4]:
from sigmaepsilon.fem import Structure, LineMesh, PointData, NodalSupport
from neumann.linalg import linspace, Vector
from neumann import repeat
from polymesh.space import StandardFrame, PointCloud
from sigmaepsilon.fem.cells import B2 as Beam
import numpy as np

# model stiffness matrix
G = E / (2 * (1 + nu))
Hooke = np.array(
    [[E * A, 0, 0, 0], [0, G * Ix, 0, 0], [0, 0, E * Iy, 0], [0, 0, 0, E * Iz]]
)

# space
GlobalFrame = StandardFrame(dim=3)

# mesh
p0 = np.array([0.0, 0.0, 0.0])
p1 = np.array([L, 0.0, 0.0])
coords = linspace(p0, p1, nElem + 1)
coords = PointCloud(coords, frame=GlobalFrame).show()
topo = np.zeros((nElem, 2), dtype=int)
topo[:, 0] = np.arange(nElem)
topo[:, 1] = np.arange(nElem) + 1

# load at the rightmost node in Y and Z directions
nodal_loads = np.zeros((coords.shape[0], 6, 3))
global_load_vector = Vector([F, 0, 0.0], frame=GlobalFrame).show()
nodal_loads[-1, :3, 0] = global_load_vector
global_load_vector = Vector([0.0, F, 0.0], frame=GlobalFrame).show()
nodal_loads[-1, :3, 1] = global_load_vector
global_load_vector = Vector([0.0, 0, F], frame=GlobalFrame).show()
nodal_loads[-1, :3, 2] = global_load_vector

# support at the leftmost node (all degrees)
fixity = np.zeros((coords.shape[0], 6)).astype(bool)
fixity[0, :] = True
fixity = fixity.astype(float) * 1e12
# support at the rightmost node
support = NodalSupport(x=[L, 0, 0], UX=0.0, UY=0.0, UZ=0.0, UXX=0.0, UYY=0.0, UZZ=0.0)

# mass and density
nodal_masses = np.zeros((coords.shape[0],))
densities = np.full((topo.shape[0],), dpa)

# pointdata
pd = PointData(coords=coords, loads=nodal_loads, fixity=fixity, mass=nodal_masses)

# cell fixity / end releases
cell_fixity = np.full((topo.shape[0], topo.shape[1], 6), True)
cell_fixity[int(nElem / 2 - 1), -1, 4] = False

# celldata
frames = repeat(GlobalFrame.show(), topo.shape[0])
cd = Beam(
    topo=topo, frames=frames, density=densities, fixity=cell_fixity, material=Hooke
)

# set up mesh and structure
mesh = LineMesh(pd, cd, frame=GlobalFrame)
structure = Structure(mesh=mesh)
structure.constraints.append(support)

# perform vinration analysis
structure.free_vibration_analysis(normalize=True, as_dense=True)
freqs, modes = structure.natural_circular_frequencies(return_vectors=True)

## Effective Modal Masses

In [5]:
from sigmaepsilon.fem.dyn import effective_modal_masses

nN, nD = len(coords), 6

action_x = np.zeros((nN, nD))
action_x[:, 0] = 1.0
action_x = action_x.reshape(nN * nD)
action_y = np.zeros((nN, nD))
action_y[:, 1] = 1.0
action_y = action_y.reshape(nN * nD)
action_z = np.zeros((nN, nD))
action_z[:, 2] = 1.0
action_z = action_z.reshape(nN * nD)
actions = np.stack([action_x, action_y, action_z], axis=1)

M = structure.mass_matrix(penalize=False)
m_eff = effective_modal_masses(M, actions, modes)

print_modal_table(m_eff, freqs, 10)

+----+--------+--------+--------+--------+
|    |   mx   |   my   |   mz   |  freq  |
+====+========+========+========+========+
| 13 | 0.000  | 60.154 | 0.000  | 2.362  |
+----+--------+--------+--------+--------+
| 14 | 0.000  | 0.000  | 51.652 | 4.705  |
+----+--------+--------+--------+--------+
| 16 | 0.000  | 11.506 | 0.000  | 12.740 |
+----+--------+--------+--------+--------+
| 19 | 0.000  | 0.000  | 15.923 | 29.089 |
+----+--------+--------+--------+--------+
| 21 | 70.671 | 0.000  | 0.000  | 34.511 |
+----+--------+--------+--------+--------+


## Plotting with Matplotlib

In [6]:
%matplotlib qt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from latexdocs.utils import floatformatter

f2s = floatformatter(sig=4)

fps = 120
i = 18

omega = freqs[i]
T = 2 * np.pi / omega
u = mesh.pd.vshapes[:, :, i] * 0.1
x = mesh.pd.x[:, 0]
dt = T / fps
f = 1 / T

fig, ax = plt.subplots()
title = "f : {f} [1/s]    T : {T} [s]".format(f=f2s.format(f), T=f2s.format(T))
ax.set_title(title)
(line,) = ax.plot(x, u[:, 2] * np.sin(omega * 0))


def init():
    line.set_ydata(u[:, 2] * np.sin(omega * 0))
    return (line,)


def animate(i):
    t = i * dt
    line.set_ydata(u[:, 2] * np.sin(omega * t))  # update the data
    return (line,)


ani = animation.FuncAnimation(
    fig, animate, fps, init_func=init, interval=1000 * dt, blit=True
)

plt.show()